In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import numpy as np
from scipy import stats
import param
import panel as pn
pn.extension('tabulator')
pn.extension('plotly')

import hvplot.pandas
import plotly.express as px
# hvplot.extension('plotly')


In [6]:
df = pd.read_csv('dashboard_data.csv')
df = df.drop(columns='Unnamed: 0')
df.sample(10)

,Date,Amplitude,Time,Temperature,Dew_Point,Humidity,Wind,Wind_Speed,Pressure,Condition,Region,Month,Year
2340,2019-01-17,21.666667,8:00 PM,20.000000,18.888889,94.0,NW,5.0,30.07,Light Rain,Da Nang,1,2019
202200,2022-11-16,10.000000,9:30 PM,26.111111,22.222222,78.0,SE,10.0,29.84,Fair,Ha Noi,11,2022
64714,2020-03-29,13.888889,12:30 PM,25.000000,22.777778,89.0,SE,9.0,29.72,Light Drizzle,Ha Noi,3,2020
136861,2021-08-18,15.000000,5:30 PM,32.222222,25.000000,66.0,ESE,8.0,29.64,Fair,Da Nang,8,2021
69907,2020-05-05,13.333333,11:00 AM,33.888889,25.000000,59.0,VAR,3.0,29.76,Mostly Cloudy,Sai Gon,5,2020
86176,2020-08-27,8.888889,8:30 AM,30.000000,27.222222,84.0,WSW,12.0,29.70,Partly Cloudy,Sai Gon,8,2020
52891,2020-01-07,15.000000,8:30 AM,27.222222,21.111111,70.0,VAR,1.0,30.03,Fair,Da Nang,1,2020
80903,2020-07-21,10.000000,12:00 AM,26.111111,25.000000,94.0,W,1.0,29.73,Partly Cloudy,Sai Gon,7,2020
32194,2019-08-15,10.555556,2:00 PM,36.111111,26.111111,56.0,VAR,2.0,29.51,Fair,Ha Noi,8,2019
90533,2020-09-26,11.111111,10:00 PM,27.777778,25.000000,84.0,SW,2.0,29.76,Fair,Da Nang,9,2020


In [8]:
df.columns

Index(['Date', 'Amplitude', 'Time', 'Temperature', 'Dew_Point', 'Humidity',
       'Wind', 'Wind_Speed', 'Pressure', 'Condition', 'Region', 'Month',
       'Year'],
      dtype='object')

## Biên độ nhiệt

In [7]:
# df_new = df.groupby("Date")['Temperature'].max
# df['Amplitude'] = df['Temperature'] - df['Temperature']

In [8]:
# biên độ nhiệt
# amp = pd.DataFrame(df.groupby("Date")['Temperature'].max() - df.groupby("Date")['Temperature'].min())
# amp.columns = ['Amplitude']
# fig = px.line(amp, x=amp.index,y='Amplitude', title='Hướng gió', width=500, height=600 )
# fig.show()

In [9]:
# dfa = pd.DataFrame(df['Wind'].value_counts())
# # dfa.columns = ["name","value"]
# fig = px.pie(dfa,values='Wind', title='Hướng gió', names=dfa.index, width=500,
#     height=600 )
# fig.show()

## DashBoard

In [6]:
# Years = list(set(df['Year']))
# Years[1]

In [7]:
# dfa = df[df['Region']== 'Da Nang']
# dfa[dfa['Year']== 2020]

In [12]:
# fig = px.bar(dfc,x='Condition', title='Hướng gió', y=dfc.index, width=500, height=600)
# fig.show()

In [16]:
# set(df['year'])
Years = list(set(df['Year']))
Regions = list(set(df['Region']))
inputs = ['Month','Temperature','Dew_Point', 'Humidity','Year']

class Dashboard(param.Parameterized):
    Region = param.Selector(Regions, default= Regions[1])
    select = param.Selector(Years, default= Years[1])
    X_variable = param.Selector(inputs, default=inputs[2])
    Y_variable = param.Selector(inputs, default=inputs[1])
    
    @param.depends('X_variable', 'Y_variable', 'select', 'Region')
    def scatter_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.scatter(x=self.X_variable, y=self.Y_variable, width=500, height=450)
    
    @param.depends('Y_variable', 'select', 'Region')
    def line_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.line('Date', self.Y_variable, width=1500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def violin_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].hvplot.violin(self.Y_variable, 'Month', width=1500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def subline_plot(self):
        dfa = df[df['Region']==self.Region]
        return  dfa[dfa['Year']== self.select].groupby('Month').mean().hvplot(x='Month', y=['Temperature', 'Dew_Point', 'Humidity'],
                 value_label='Mean', subplots=True, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def box_plot(self):
        dfa = df[df['Region']==self.Region]
        df_box = dfa[dfa['Year']== self.select].groupby('Date').max()
        return df_box.hvplot.box(by='Month', y= self.Y_variable, legend=False, width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def pie_plot(self):
        dfb = df[df['Region']==self.Region]
        dfa = pd.DataFrame(dfb[dfb['Year']== self.select]['Wind'].value_counts())
        fig = px.pie(dfa,values='Wind', names=dfa.index)
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('Y_variable', 'select', 'Region')
    def barh_condition(self):
        dfa = df[df['Region']==self.Region]
        dfc = pd.DataFrame(dfa[dfa['Year']== self.select].groupby('Condition').count()).sort_values('Time')[10:]
        return dfc.hvplot.barh(y='Time', width=500)
    
    @param.depends('select', 'Region')
    def amp_plot(self):
        dfa = df[df['Region']==self.Region]
        amp = pd.DataFrame(dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].max() - dfa[dfa['Year']== self.select].groupby("Date")['Temperature'].min())
        amp.columns = ['Amplitude']
        return amp.hvplot.line('Date', self.Y_variable, width=1500)
        # fig = px.line(amp, x=amp.index,y='Amplitude', title='Hướng gió')
        # return pn.pane.Plotly(fig, config={'responsive': True}, width=1500)
    
    @param.depends('select', 'Region')
    def heat_map(self):
        dfa = df[df['Region'] == self.Region]
        z =dfa[dfa['Year']== self.select][['Temperature', 'Dew_Point', 'Humidity', 'Pressure']].corr()
        fig = px.imshow(z, text_auto=True,  color_continuous_scale='Blues')
        return pn.pane.Plotly(fig, config={'responsive': True} , width=500)
    
    @param.depends('select', 'Region')
    def win_speed(self):
        dfa = df[df['Region']==self.Region]
        wind = dfa[dfa['Year']== self.select].groupby('Wind_Speed').count()
        return wind.hvplot.bar(y='Time', width=500)
            
        
         
        
    
    def panel(self):
        return pn.Row(
            pn.Param(self, width=200, name = "Plot Settings", sizing_mode="fixed"),
            pn.Column(
                self.line_plot,
                self.subline_plot,
                pn.Row(self.heat_map,self.pie_plot, self.scatter_plot),
                pn.Row(self.win_speed, self.box_plot,self.barh_condition),
            # self.amp_plot,
            # self.subline_plot,
            # self.box_plot,
            # self.pie_plot,
            # self.scatter_plot,
            # self.violin_plot
            # self.line_plot,
            # self.barh_condition,
            # self.heat_map,
            # self.win_speed,self.barh_condition
            )
        )
dashboard = Dashboard(name='Weather_Dashboard')
component = dashboard.panel()
component

Row
    [0] Param(Dashboard, name='Plot Settings', sizing_mode='fixed', width=200)
    [1] Column
        [0] ParamMethod(method)
        [1] ParamMethod(method)
        [2] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)
        [3] Row
            [0] ParamMethod(method)
            [1] ParamMethod(method)
            [2] ParamMethod(method)

In [69]:
pn.template.FastListTemplate(site="Panel", title="Weather_Dashboard", 
                             main=[
                                 component 
                             ]).servable();

In [ ]:
# df['Month'] = pd.DatetimeIndex(df['Date']).month.astype(str)
# df['year'] = pd.DatetimeIndex(df['Date']).year.astype(str)
# df_box = df.groupby('Date').max()
# sns.boxplot(data = df_box, x ="Month", y="Dew_Point")
# plt.title('Daily Maximum Dew Point Observations')